In [224]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

%matplotlib inline

In [225]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import precision_score, \
    recall_score, confusion_matrix, classification_report, \
    accuracy_score, f1_score

In [261]:
df = pd.read_csv('preptraindata.csv')

In [262]:
df.head()

,PassengerId,Survived,Age,SibSp,Parch,Fare,OrderPclass,FamilySize,Sex_female,Sex_male,...,Title_Rev.,CabinZone_A,CabinZone_B,CabinZone_C,CabinZone_D,CabinZone_E,CabinZone_F,CabinZone_G,CabinZone_None,CabinZone_T
0,1,0,22.0,1,0,7.2500,1.0,1,0,1,...,0,0,0,0,0,0,0,0,1,0
1,2,1,38.0,1,0,71.2833,3.0,1,1,0,...,0,0,0,1,0,0,0,0,0,0
2,3,1,26.0,0,0,7.9250,1.0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,4,1,35.0,1,0,53.1000,3.0,1,1,0,...,0,0,0,1,0,0,0,0,0,0
4,5,0,35.0,0,0,8.0500,1.0,0,0,1,...,0,0,0,0,0,0,0,0,1,0


In [263]:
# For Train Data only
X = df.drop(['Survived'],axis=1)
X = X.drop(['PassengerId'],axis=1)
Y = df['Survived']

In [264]:
X_Train, X_Test, Y_Train, Y_Test = train_test_split(X, Y, test_size=0.33)

In [256]:
# #For test
# X_Test = df.drop(['PassengerId'],axis=1)

## XGBoost

In [2]:
!pip install xgboost

twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [232]:
import xgboost as xgb

In [233]:
from xgboost.sklearn import XGBClassifier

In [296]:
#gbm = xgb.XGBClassifier(n_estimators=300, learning_rate=0.05, max_depth= 7, min_child_weight= 3, gamma= 0.4,reg_alpha=0).fit(X_Train, Y_Train)

In [299]:
gbm = xgb.XGBClassifier(n_estimators=300, learning_rate=0.01, max_depth= 7, min_child_weight= 3, gamma= 0.4,reg_alpha=0).fit(X_Train, Y_Train)

In [300]:
predictions = gbm.predict(X_Test)

C:\Users\teing\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [301]:
print(classification_report(Y_Test,predictions))

             precision    recall  f1-score   support

          0       0.82      0.96      0.89       177
          1       0.92      0.69      0.79       118

avg / total       0.86      0.85      0.85       295



In [302]:
print('Accuracy:', accuracy_score(Y_Test, predictions))

Accuracy: 0.8508474576271187


## Parameters Tuning 

In [178]:
from sklearn.grid_search import GridSearchCV   #Perforing grid search

C:\Users\teing\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\teing\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


### Step1 Param Tuning 

#### Fix learning rate and number of estimators for tuning tree-based parameters

In [190]:
def modelfit(alg, dtrain, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds, show_progress=False)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain['Disbursed'],eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
        
    #Print model report:
    print("\nModel Report")
    print("Accuracy : %.4g" % metrics.accuracy_score(dtrain['Disbursed'].values, dtrain_predictions))
    print("AUC Score (Train): %f" % metrics.roc_auc_score(dtrain['Disbursed'], dtrain_predprob))
                    
    feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')

In [192]:
predictors = [x for x in X_Train.columns]
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
modelfit(xgb1, X_Train, predictors)

NameError: name 'target' is not defined

### Step 2: Tune max_depth and min_child_weight 

In [265]:
predictors = [x for x in X_Train.columns]

In [266]:
param_test1 = {
 'max_depth':np.arange(3,10,2),
 'min_child_weight':np.arange(1,6,2)
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='accuracy',n_jobs=4,iid=False, cv=5)
gsearch1.fit(X_Train,Y_Train)

gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

([mean: 0.81022, std: 0.03644, params: {'max_depth': 3, 'min_child_weight': 1},
  mean: 0.80351, std: 0.03362, params: {'max_depth': 3, 'min_child_weight': 3},
  mean: 0.81022, std: 0.03528, params: {'max_depth': 3, 'min_child_weight': 5},
  mean: 0.81358, std: 0.03179, params: {'max_depth': 5, 'min_child_weight': 1},
  mean: 0.80018, std: 0.03085, params: {'max_depth': 5, 'min_child_weight': 3},
  mean: 0.80520, std: 0.03389, params: {'max_depth': 5, 'min_child_weight': 5},
  mean: 0.81356, std: 0.03723, params: {'max_depth': 7, 'min_child_weight': 1},
  mean: 0.80852, std: 0.03758, params: {'max_depth': 7, 'min_child_weight': 3},
  mean: 0.80349, std: 0.03655, params: {'max_depth': 7, 'min_child_weight': 5},
  mean: 0.80852, std: 0.03532, params: {'max_depth': 9, 'min_child_weight': 1},
  mean: 0.80518, std: 0.03761, params: {'max_depth': 9, 'min_child_weight': 3},
  mean: 0.80519, std: 0.03436, params: {'max_depth': 9, 'min_child_weight': 5}],
 {'max_depth': 5, 'min_child_weight': 1

In [269]:
param_test2 = {
 'max_depth':[4,5,6],
 'min_child_weight':[0,0.5,1]
}
gsearch2 = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=5,
 min_child_weight=2, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test2, scoring='accuracy',n_jobs=4,iid=False, cv=5)
gsearch2.fit(X_Train,Y_Train)
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

([mean: 0.81022, std: 0.03233, params: {'max_depth': 4, 'min_child_weight': 0},
  mean: 0.81525, std: 0.03583, params: {'max_depth': 4, 'min_child_weight': 0.5},
  mean: 0.80683, std: 0.04124, params: {'max_depth': 4, 'min_child_weight': 1},
  mean: 0.81690, std: 0.03693, params: {'max_depth': 5, 'min_child_weight': 0},
  mean: 0.81520, std: 0.03987, params: {'max_depth': 5, 'min_child_weight': 0.5},
  mean: 0.81358, std: 0.03179, params: {'max_depth': 5, 'min_child_weight': 1},
  mean: 0.80681, std: 0.04055, params: {'max_depth': 6, 'min_child_weight': 0},
  mean: 0.80520, std: 0.02950, params: {'max_depth': 6, 'min_child_weight': 0.5},
  mean: 0.81186, std: 0.03898, params: {'max_depth': 6, 'min_child_weight': 1}],
 {'max_depth': 5, 'min_child_weight': 0},
 0.8168978777951859)

### Step 3: Tune gamma

In [270]:
param_test3 = {
 'gamma':[i/10.0 for i in range(0,5)]
}
gsearch3 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=0, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test3, scoring='accuracy',n_jobs=4,iid=False, cv=5)
gsearch3.fit(X_Train,Y_Train)
gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

([mean: 0.81690, std: 0.03693, params: {'gamma': 0.0},
  mean: 0.80852, std: 0.03683, params: {'gamma': 0.1},
  mean: 0.81191, std: 0.03045, params: {'gamma': 0.2},
  mean: 0.81356, std: 0.03528, params: {'gamma': 0.3},
  mean: 0.81693, std: 0.03567, params: {'gamma': 0.4}],
 {'gamma': 0.4},
 0.8169258889996677)

### Step 4: Tune subsample and colsample_bytree

In [272]:
param_test4 = {
 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch4 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=0, gamma=0.4, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test4, scoring='accuracy',n_jobs=4,iid=False, cv=5)
gsearch4.fit(X_Train,Y_Train)
gsearch4.grid_scores_, gsearch4.best_params_, gsearch4.best_score_

([mean: 0.81691, std: 0.03521, params: {'colsample_bytree': 0.6, 'subsample': 0.6},
  mean: 0.81186, std: 0.03884, params: {'colsample_bytree': 0.6, 'subsample': 0.7},
  mean: 0.81526, std: 0.03073, params: {'colsample_bytree': 0.6, 'subsample': 0.8},
  mean: 0.81351, std: 0.04211, params: {'colsample_bytree': 0.6, 'subsample': 0.9},
  mean: 0.81861, std: 0.03097, params: {'colsample_bytree': 0.7, 'subsample': 0.6},
  mean: 0.81852, std: 0.04388, params: {'colsample_bytree': 0.7, 'subsample': 0.7},
  mean: 0.81191, std: 0.03176, params: {'colsample_bytree': 0.7, 'subsample': 0.8},
  mean: 0.81520, std: 0.03872, params: {'colsample_bytree': 0.7, 'subsample': 0.9},
  mean: 0.81856, std: 0.04096, params: {'colsample_bytree': 0.8, 'subsample': 0.6},
  mean: 0.81349, std: 0.04412, params: {'colsample_bytree': 0.8, 'subsample': 0.7},
  mean: 0.81693, std: 0.03567, params: {'colsample_bytree': 0.8, 'subsample': 0.8},
  mean: 0.81184, std: 0.04148, params: {'colsample_bytree': 0.8, 'subsample'

In [274]:
param_test5 = {
 'subsample':[i/100.0 for i in range(50,70,5)],
 'colsample_bytree':[i/100.0 for i in range(60,80,5)]
}
gsearch5 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=0, gamma=0.4, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test4, scoring='accuracy',n_jobs=4,iid=False, cv=5)
gsearch5.fit(X_Train,Y_Train)
gsearch5.grid_scores_, gsearch4.best_params_, gsearch4.best_score_

([mean: 0.81691, std: 0.03521, params: {'colsample_bytree': 0.6, 'subsample': 0.6},
  mean: 0.81186, std: 0.03884, params: {'colsample_bytree': 0.6, 'subsample': 0.7},
  mean: 0.81526, std: 0.03073, params: {'colsample_bytree': 0.6, 'subsample': 0.8},
  mean: 0.81351, std: 0.04211, params: {'colsample_bytree': 0.6, 'subsample': 0.9},
  mean: 0.81861, std: 0.03097, params: {'colsample_bytree': 0.7, 'subsample': 0.6},
  mean: 0.81852, std: 0.04388, params: {'colsample_bytree': 0.7, 'subsample': 0.7},
  mean: 0.81191, std: 0.03176, params: {'colsample_bytree': 0.7, 'subsample': 0.8},
  mean: 0.81520, std: 0.03872, params: {'colsample_bytree': 0.7, 'subsample': 0.9},
  mean: 0.81856, std: 0.04096, params: {'colsample_bytree': 0.8, 'subsample': 0.6},
  mean: 0.81349, std: 0.04412, params: {'colsample_bytree': 0.8, 'subsample': 0.7},
  mean: 0.81693, std: 0.03567, params: {'colsample_bytree': 0.8, 'subsample': 0.8},
  mean: 0.81184, std: 0.04148, params: {'colsample_bytree': 0.8, 'subsample'

### Step 5: Tuning Regularization Parameters

In [275]:
param_test6 = {
 'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
}
gsearch6 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=0, gamma=0.4, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test6, scoring='accuracy',n_jobs=4,iid=False, cv=5)
gsearch6.fit(X_Train,Y_Train)
gsearch6.grid_scores_, gsearch6.best_params_, gsearch6.best_score_

([mean: 0.81188, std: 0.03373, params: {'reg_alpha': 1e-05},
  mean: 0.81526, std: 0.03249, params: {'reg_alpha': 0.01},
  mean: 0.81358, std: 0.03352, params: {'reg_alpha': 0.1},
  mean: 0.80349, std: 0.03873, params: {'reg_alpha': 1},
  mean: 0.62416, std: 0.00204, params: {'reg_alpha': 100}],
 {'reg_alpha': 0.01},
 0.8152594597160899)

In [277]:
param_test7 = {
 'reg_alpha':[0, 0.001, 0.005, 0.01, 0.05]
}
gsearch7 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=0, gamma=0.4, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test7, scoring='accuracy',n_jobs=4,iid=False, cv=5)
gsearch7.fit(X_Train,Y_Train)
gsearch7.grid_scores_, gsearch7.best_params_, gsearch7.best_score_

([mean: 0.81693, std: 0.03567, params: {'reg_alpha': 0},
  mean: 0.80690, std: 0.02765, params: {'reg_alpha': 0.001},
  mean: 0.80687, std: 0.03253, params: {'reg_alpha': 0.005},
  mean: 0.81526, std: 0.03249, params: {'reg_alpha': 0.01},
  mean: 0.81187, std: 0.03705, params: {'reg_alpha': 0.05}],
 {'reg_alpha': 0},
 0.8169258889996677)

### Step 6: Reducing Learning Rate

### For Test

In [220]:
X_Test.columns

Index(['Age', 'SibSp', 'Parch', 'Fare', 'Sex_male', 'Embarked_Q', 'Embarked_S',
       'Title_Army.', 'Title_Dr.', 'Title_Master.', 'Title_Miss.', 'Title_Mr.',
       'Title_Mrs.', 'Title_Noble.', 'Title_Rev.', 'hasCabin', 'CabinZone_A',
       'CabinZone_B', 'CabinZone_C', 'CabinZone_D', 'CabinZone_E',
       'CabinZone_F', 'CabinZone_G', 'CabinZone_None', 'CabinZone_T',
       'OrderPclass'],
      dtype='object')

In [258]:
df['Survived']=predictions

In [259]:
df =df[['PassengerId','Survived']]

In [260]:
df.to_csv('prediction_result.csv',index = False)